In [17]:
import numpy as np
import asset_calculations
import bond_calculations

In [23]:
nominal = 100
coupon = 0.04
ytm = 0.045
maturity = 8
freq = 2

In [24]:
bond_calculations.bond_price(nominal=nominal, coupon=coupon, ytm=ytm, maturity=maturity, freq=2)

96.67184217326447

In [25]:
bond_calculations.bond_duration(nominal=nominal, coupon=coupon, ytm=ytm, maturity=maturity, freq=2)

(13.804407676783661, 13.500643204678397)

In [26]:
bond_calculations.bond_convexity(nominal=nominal, coupon=coupon, ytm=ytm, maturity=maturity, freq=2)

53.32576355984632

In [27]:
bond_calculations.bond_ytm(nominal=nominal, market_price=82, coupon=coupon, maturity=maturity)

0.06868131868131869